
# БЛОК 1: Импорт библиотек и загрузка данных
 В этом блоке мы подключаем библиотеки для работы с табличными данными,
 числовыми массивами и текстовыми признаками, а также наш модуль utils
 для визуализации. Затем считываем CSV-файл 'IMDB_Dataset.csv' в
 pandas DataFrame и выводим первые строки, чтобы убедиться в корректности загрузки.


In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import utils

# Читаем готовый датасет
movies = pd.read_csv('IMDB_Dataset.csv')  # файл IMDB_Dataset.csv в рабочей папке
# Выводим первые 5 строк для проверки структуры
print("Первые 5 строк исходного датасета:")
print(movies.head())




Первые 5 строк исходного датасета:
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive



# Преобразование текстовых отзывов в числовые векторные признаки
Машинное обучение не умеет работать с необработанным текстом. Поэтому:
 1. Используем CountVectorizer, который преобразует каждый отзыв в вектор:
    индекс i соответствует слову i в словаре, а значение — количество
    вхождений этого слова в отзыве.
 2. Получаем разреженную матрицу X и массив меток y.


In [3]:
vectorizer = CountVectorizer()
# Строим словарь и преобразуем тексты в матрицу признаков
X = vectorizer.fit_transform(movies['review'])  # shape (n_samples, n_features)
# Кодируем метки "positive"→1, "negative"→0
y = (movies['sentiment'] == 'positive').astype(int).values

print(f"Размерность матрицы признаков: {X.shape}")  # (число примеров, число уникальных слов)

Размерность матрицы признаков: (50000, 101895)


# БЛОК 3: Обучение логистической регрессии на TF (term-frequency) признаках
 Делим данные на обучающую и тестовую выборки, затем обучаем
 sklearn LogisticRegression. Выводим точность на train и test.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = LogisticRegression(max_iter=1000, solver='liblinear')
model.fit(X_train, y_train)

train_acc = accuracy_score(y_train, model.predict(X_train))
test_acc = accuracy_score(y_test, model.predict(X_test))
print(f"Accuracy на обучении: {train_acc:.4f}")
print(f"Accuracy на тесте:    {test_acc:.4f}")



Accuracy на обучении: 0.9972
Accuracy на тесте:    0.8841



# БЛОК 4: Анализ весов модели (коэффициентов)
LogisticRegression обучает вес для каждого слова в словаре.
Положительный вес означает, что слово увеличивает вероятность
 положительного отзыва, отрицательный — снижает.
 В этом блоке:
 1. Собираем DataFrame со словами и их коэффициентами.
 2. Сортируем по возрастанию и убыванию.
 3. Извлекаем веса для конкретных слов: "wonderful", "horrible", "the".

In [5]:
# Получаем имена признаков (слова)
feature_names = vectorizer.get_feature_names_out()
# Получаем массив коэффициентов (shape (n_features,))
coefs = model.coef_[0]
# Собираем DataFrame для анализа
weights_df = pd.DataFrame({'word': feature_names, 'coef': coefs})

# Сортировка по коэффициенту
print("\n10 самых отрицательных слов (наиболее указывают на negative):")
print(weights_df.nsmallest(10, 'coef'))
print("\n10 самых положительных слов (наиболее указывают на positive):")
print(weights_df.nlargest(10, 'coef'))

# Конкретные слова
for w in ['wonderful', 'horrible', 'the']:
    val = weights_df.loc[weights_df['word'] == w, 'coef']
    print(f"coef для '{w}':", float(val) if not val.empty else 'слово не в словаре')


10 самых отрицательных слов (наиболее указывают на negative):
                  word      coef
98190            waste -2.208487
100223           worst -2.193702
34586      forgettable -1.889602
25426    disappointing -1.725044
7242             awful -1.701960
25428   disappointment -1.697313
86003          stinker -1.646192
27196         dreadful -1.599458
31913            fails -1.575412
94846    uninteresting -1.572674

10 самых положительных слов (наиболее указывают на positive):
              word      coef
74052   refreshing  1.600284
42850       hooked  1.459614
87465       superb  1.442454
2675           adr  1.413679
73132      raunchy  1.403377
99949  wonderfully  1.398362
35704     funniest  1.390787
85147     squirrel  1.357860
69609      portman  1.330558
23671   delightful  1.303313
coef для 'wonderful': 0.916165310227916
coef для 'horrible': -1.5082530762312658
coef для 'the': 0.014568460574303933


/tmp/ipykernel_14714/2781771247.py:17: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  print(f"coef для '{w}':", float(val) if not val.empty else 'слово не в словаре')


# Предсказания вероятностей и их анализ
 LogisticRegression может выдавать вероятность принадлежности к классу 1.
 Добавляем столбец 'prob_positive' в исходный DataFrame и показываем
 самый уверенный positive и самый уверенный negative отзыв.

In [6]:
probs = model.predict_proba(X)[:,1]  # вероятность класса 1
movies['prob_positive'] = probs

# Самый уверенный положительный
best_pos = movies.loc[movies['prob_positive'].idxmax()]
# Самый уверенный отрицательный
best_neg = movies.loc[movies['prob_positive'].idxmin()]

print("\nОтзыв с наибольшей вероятностью positive:")
print(best_pos[['review','sentiment','prob_positive']])
print("\nОтзыв с наибольшей уверенностью negative:")
print(best_neg[['review','sentiment','prob_positive']])


Отзыв с наибольшей вероятностью positive:
review           A film for mature, educated audiences...<br />...
sentiment                                                 positive
prob_positive                                                  1.0
Name: 943, dtype: object

Отзыв с наибольшей уверенностью negative:
review           Zombi 3 starts as a group of heavily armed men...
sentiment                                                 negative
prob_positive                                                  0.0
Name: 13452, dtype: object
